In [1]:
!pip install pytreebank


  Created wheel for pytreebank: filename=pytreebank-0.2.7-py3-none-any.whl size=37071 sha256=3369f26ca14bcbb2c614004b648f42db767b334df3b0fe6aa9fc415001acdc22
  Stored in directory: /home/ubuntu/.cache/pip/wheels/a1/37/4d/a489a8c9045c55f783a3d49ee2354484e4c81c3414c0e4849a
Successfully built pytreebank


In [2]:
import pytreebank
dataset = pytreebank.load_sst()
example = dataset["train"][0]

# extract spans from the tree.
for label, sentence in example.to_labeled_lines():
    print("%s has sentiment label %s" % (sentence,["very negative", "negative", "neutral", "positive", "very positive"][label]))

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal . has sentiment label positive
The Rock has sentiment label neutral
The has sentiment label neutral
Rock has sentiment label neutral
is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal . has sentiment label very positive
is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal has sentiment label positive
is has sentiment label neutral
destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal has sentiment label very positive
destined has sentiment label neutral
to be the 

In [3]:
[key for key in dataset.keys()]

['train', 'test', 'dev']

In [4]:
example

In [20]:
import pandas as pd
df = pd.DataFrame(columns=["Sentence","Label","Dataset"])
df = df.astype(dtype={'Sentence':str,'Label':int,'Dataset':str})

for key_value in dataset.keys():
    for i,lbl_tree in enumerate(dataset[key_value]):
        #print(lbl_tree.to_labeled_lines()[0])
        label,sentence = lbl_tree.to_labeled_lines()[0]
        df = df.append({"Sentence":sentence,"Label":label,"Dataset":key_value}, ignore_index=True)

In [21]:
print("Length of the Dataset: ",len(df))
df.Label.value_counts()

Length of the Dataset:  11855


1    3140
3    3111
2    2242
4    1852
0    1510
Name: Label, dtype: int64

In [22]:
df.to_csv("dataset/sst_dataset.csv")
for key in dataset.keys():
    df_ = df[df["Dataset"]==key]
    df_.to_csv("dataset/" + key+".csv")
    print("Number of sentences in "+key+ ": ",len(df_))

Number of sentences in train:  8544
Number of sentences in test:  2210
Number of sentences in dev:  1101


In [23]:
!pip install nltk
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
### Data Augmentation

import random
import nltk
from nltk.corpus import wordnet 
random.seed(50)

#stop words list
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
			'ours', 'ourselves', 'you', 'your', 'yours', 
			'yourself', 'yourselves', 'he', 'him', 'his', 
			'himself', 'she', 'her', 'hers', 'herself', 
			'it', 'its', 'itself', 'they', 'them', 'their', 
			'theirs', 'themselves', 'what', 'which', 'who', 
			'whom', 'this', 'that', 'these', 'those', 'am', 
			'is', 'are', 'was', 'were', 'be', 'been', 'being', 
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at', 
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after', 
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again', 
			'further', 'then', 'once', 'here', 'there', 'when', 
			'where', 'why', 'how', 'all', 'any', 'both', 'each', 
			'few', 'more', 'most', 'other', 'some', 'such', 'no', 
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
			'very', 's', 't', 'can', 'will', 'just', 'don', 
			'should', 'now', '']


def random_deletion(words, p=0.3): #sentence: list of words/tokens from a sentence
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

def random_swap(sentence, n=5): #sentence: list of words/tokens from a sentence
    sen_len = len(sentence)
    length = range(sen_len)
    if (sen_len>1): 
        for _ in range(n):
            idx1, idx2 = random.sample(length, 2)
            sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
        return sentence
    else:
        return sentence


def synonym_replacement(words, n=5):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)


In [27]:
import pandas as pd
from sklearn.utils import shuffle

df = pd.read_csv("dataset/train.csv")
df = shuffle(df)
df.reset_index(inplace=True, drop=True)

#df_aug = pd.DataFrame(columns=["Sentence","Label","Dataset"])
#df_aug = df.astype(dtype={'Sentence':str,'Label':int,'Dataset':str})

In [28]:
import re

def clean_text(text):
    text = text.replace("`","'")
    text = re.sub(r"[-](LRB|RRB|RSB|LSB)[-]","",text)
    text = re.sub(r"[^0-9a-zA-Z',\-\.]"," ",text)
    text = " ".join([t for t in text.split(" ") if t is not ""])
    return text

In [29]:
clean_text("""The Rock is destined to be the 21st Century 's new `` Conan '' and 
           that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .""")

"The Rock is destined to be the 21st Century 's new '' Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal ."

In [30]:
#Wrapper Functions
def synonym_replace_aug(text):
    return " ".join(synonym_replacement(text.split(" "),n=4))
def random_delete_aug(text):
    return " ".join(random_deletion(text.split(" ")))
def random_swap_aug(text):
    return " ".join(random_swap(text.split(" ")))

In [32]:
import time 
start_time = time.time()

df_sr = df[:2500].sample(1200).copy()
df_sr["Sentence"] = df_sr["Sentence"].apply(clean_text).apply(synonym_replace_aug)

df_rd = df[2501:4500].sample(900).copy()
df_rd["Sentence"] = df_rd["Sentence"].apply(clean_text).apply(random_delete_aug)

df_rs = df[4500:6000].sample(400).copy()
df_rs["Sentence"] = df_rs["Sentence"].apply(clean_text).apply(random_swap_aug)

print("Time taken : ", time.time() - start_time)
df_aug =  pd.concat([df_sr,df_rd,df_rs])
df_aug.to_csv("dataset/aug_df.csv")

Time taken :  0.42549633979797363


In [38]:
#Example for SR
print(df.loc[10,"Sentence"])
print(df_sr.loc[10,"Sentence"])

Below may not mark Mr. Twohy 's emergence into the mainstream , but his promise remains undiminished .
Below may not score Mr. Twohy 's outgrowth into the mainstream , but his hope continue undiminished .


In [10]:
#Back Translate
!pip install google-trans-new


In [17]:
df[6000:].to_csv("dataset/back_trans_candidates.csv")

In [ ]:
import multiprocessing
import pandas as pd
import numpy as np
import time
from google_trans_new import google_translator  

avbl_langs = ['af', 'sq', 'am', 'ar', 'hy', 'az', 'eu', 'be', 'bn', 'bs', 'bg', 'ca', 'ceb', 'ny', 'zh-cn', 'zh-tw', 'co', 'hr', 'cs', 'da', 'nl', 'en', 'eo', 'et', 'tl', 'fi', 'fr', 'fy', 'gl', 'ka', 'de', 'el', 'gu', 'ht', 'ha', 'haw', 'iw', 'he', 'hi', 'hmn', 'hu', 'is', 'ig', 'id', 'ga', 'it', 'ja', 'jw', 'kn', 'kk', 'km', 'ko', 'ku', 'ky', 'lo', 'la', 'lv', 'lt', 'lb', 'mk', 'mg', 'ms', 'ml', 'mt', 'mi', 'mr', 'mn', 'my', 'ne', 'no', 'or', 'ps', 'fa', 'pl', 'pt', 'pa', 'ro', 'ru', 'sm', 'gd', 'sr', 'st', 'sn', 'sd', 'si', 'sk', 'sl', 'so', 'es', 'su', 'sw', 'sv', 'tg', 'ta', 'te', 'th', 'tr', 'uk', 'ur', 'ug', 'uz', 'vi', 'cy', 'xh', 'yi', 'yo', 'zu']

def _apply_df(args):
    df, func, num, kwargs = args
    return num, df.apply(func, **kwargs)

def apply_by_multiprocessing(df,func,**kwargs):
    workers=kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, i, kwargs) for i,d in enumerate(np.array_split(df, workers))])  
    pool.close()
    result=sorted(result,key=lambda x:x[0])
    return pd.concat([i[1] for i in result])

def square(row):
    text =row["Sentence"]
    #print("Text: ", text)
    
    translator = google_translator(timeout=5) 
    trans_lang = random.choice(avbl_langs)  
      
    translate_text = translator.translate(
      translator.translate(text,lang_tgt=trans_lang),
      lang_src = trans_lang,
      lang_tgt='en')
    
    row["Sentence"] = translate_text
    #print("Translated text : ", row["Sentence"] )
    return row

df_bt = df[6000:7000].copy()
start_time=time.time()
df_bt = apply_by_multiprocessing(df_bt, square, axis=1, workers=5) 
print("Time Taken for 1000 sentences : ", time.time()-start_time)

In [13]:
df[6010:6020].head()

,Unnamed: 0,Sentence,Label,Dataset
6010,10547,Creepy but ultimately unsatisfying thriller .,1,test
6011,4111,Though the opera itself takes place mostly ind...,2,train
6012,2027,Although devoid of objectivity and full of nos...,3,train
6013,8749,"Narc may not get an ` A ' for originality , bu...",3,test
6014,1076,An intriguing look at the French film industry...,3,train


In [15]:
display(df_bt[10:20].head())

,Unnamed: 0,Sentence,Label,Dataset
6010,10547,Creepy but eventually insufficient thrillers.,1,test
6011,4111,Although the Opera itself is carried out the r...,2,train
6012,2027,Although it is lacking objectivity and now is ...,3,train
6013,8749,"The narc may not get a `a 'for originality, bu...",3,test
6014,1076,Looking amazing in the French film industry du...,3,train


In [18]:
#tried but could not


In [6]:
import pandas as pd
from sklearn.utils import shuffle
df_train = pd.read_csv("dataset/train.csv", index_col=0)
df_aug = pd.read_csv("dataset/aug_df.csv",index_col=0)
df_bt = pd.read_csv("dataset/Back_Translation.csv",index_col=0)

In [7]:
df_back_trans = pd.DataFrame(columns=df_train.columns)
df_back_trans["Sentence"]= df_bt["to_en"]
df_back_trans["Label"]= df_bt["Label"]
df_back_trans["Dataset"]= df_bt["Dataset"]

In [8]:

df_augmented_dataset = pd.concat([df_train, df_aug,df_back_trans.sample(2000)])
print(len(df_augmented_dataset))

13044


In [9]:
df_augmented_dataset = shuffle(df_augmented_dataset)
df_augmented_dataset.reset_index(inplace=True, drop=True)

In [10]:
df_augmented_dataset.Label.value_counts()

3    3545
1    3396
2    2452
4    1985
0    1666
Name: Label, dtype: int64

In [11]:
df_augmented_dataset.to_csv("dataset/train_aug.csv")

In [12]:
df_test = pd.read_csv("dataset/test.csv")
df_dev=pd.read_csv("dataset/dev.csv")
df_test_combined = pd.concat([df_test,df_dev])
df_test_combined = shuffle(df_test_combined)
df_test_combined.reset_index(inplace=True, drop=True)
df_test_combined.to_csv("dataset/test_dev.csv")

In [14]:

#80/20 split 
train_n = len(df_augmented_dataset)
test_n = len(df_test_combined)
total = train_n + test_n
percent_train = train_n/total
percent_test = test_n/total
print(f"""Number of records \nTrain dataset : %d 
      %.2f\nTest Dataset: %d %.2f\nTotal : %d"""
      %(train_n,percent_train,test_n,percent_test,total))

Number of records 
Train dataset : 13044 0.80
Test Dataset: 3311 0.20
Total : 16355


In [13]:
df_test_combined.Label.value_counts()

1    922
3    789
2    618
4    564
0    418
Name: Label, dtype: int64